In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
!pip install chumpy

     |████████████████████████████████| 51kB 1.9MB/s 
  Created wheel for chumpy: filename=chumpy-0.70-cp37-none-any.whl size=58295 sha256=a97c1532a4146cbc81322d15ba117b1c6d1ea6e810e8b24b4f35a5d6e2914b7e
  Stored in directory: /root/.cache/pip/wheels/0a/73/91/b8dbef9746a907c8c008fc194a9691224ca3371b55f16f1dbd
Successfully built chumpy


In [11]:
import numpy as np
import pickle
import chumpy


class SMPLModel():
    def __init__(self, model_path):
        """
        SMPL model.
    
        Parameter:
        ---------
        model_path: Path to the SMPL model parameters, pre-processed by
        `preprocess.py`.
    
        """
        with open(model_path, 'rb') as f:
            params = pickle.load(f, encoding='latin1')

            self.J_regressor = params['J_regressor']
            self.weights = np.asarray(params['weights'])
            self.posedirs = np.asarray(params['posedirs'])
            self.v_template = np.asarray(params['v_template'])
            self.shapedirs = np.asarray(params['shapedirs'])
            self.faces = np.asarray(params['f'])
            self.kintree_table = np.asarray(params['kintree_table'])

        id_to_col = {
            self.kintree_table[1, i]: i for i in range(self.kintree_table.shape[1])
        }
        self.parent = {
            i: id_to_col[self.kintree_table[0, i]]
            for i in range(1, self.kintree_table.shape[1])
        }

        self.pose_shape = [24, 3]
        self.beta_shape = [10]
        self.trans_shape = [3]

        self.pose = np.zeros(self.pose_shape)
        self.beta = np.zeros(self.beta_shape)
        self.trans = np.zeros(self.trans_shape)

        self.verts = None
        self.J = None
        self.R = None
        self.G = None

        self.update()

    def set_params(self, pose=None, beta=None, trans=None):
        """
        Set pose, shape, and/or translation parameters of SMPL model. Verices of the
        model will be updated and returned.
    
        Prameters:
        ---------
        pose: Also known as 'theta', a [24,3] matrix indicating child joint rotation
        relative to parent joint. For root joint it's global orientation.
        Represented in a axis-angle format.
    
        beta: Parameter for model shape. A vector of shape [10]. Coefficients for
        PCA component. Only 10 components were released by MPI.
    
        trans: Global translation of shape [3].
    
        Return:
        ------
        Updated vertices.
    
        """
        if pose is not None:
            self.pose = pose
        if beta is not None:
            self.beta = beta
        if trans is not None:
            self.trans = trans
        self.update()
        return self.verts

    def update(self):
        """
        Called automatically when parameters are updated.
    
        """
        # how beta affect body shape
        v_shaped = self.shapedirs.dot(self.beta) + self.v_template
        # joints location
        self.J = self.J_regressor.dot(v_shaped)
        pose_cube = self.pose.reshape((-1, 1, 3))
        # rotation matrix for each joint
        self.R = self.rodrigues(pose_cube)
        I_cube = np.broadcast_to(
            np.expand_dims(np.eye(3), axis=0),
            (self.R.shape[0] - 1, 3, 3)
        )
        lrotmin = (self.R[1:] - I_cube).ravel()
        # how pose affect body shape in zero pose
        v_posed = v_shaped + self.posedirs.dot(lrotmin)
        # world transformation of each joint
        G = np.empty((self.kintree_table.shape[1], 4, 4))
        G[0] = self.with_zeros(np.hstack((self.R[0], self.J[0, :].reshape([3, 1]))))
        for i in range(1, self.kintree_table.shape[1]):
            G[i] = G[self.parent[i]].dot(
                self.with_zeros(
                    np.hstack(
                        [self.R[i], ((self.J[i, :] - self.J[self.parent[i], :]).reshape([3, 1]))]
                    )
                )
            )
        # remove the transformation due to the rest pose
        G = G - self.pack(
            np.matmul(
                G,
                np.hstack([self.J, np.zeros([24, 1])]).reshape([24, 4, 1])
            )
        )
        # transformation of each vertex
        T = np.tensordot(self.weights, G, axes=[[1], [0]])
        rest_shape_h = np.hstack((v_posed, np.ones([v_posed.shape[0], 1])))
        v = np.matmul(T, rest_shape_h.reshape([-1, 4, 1])).reshape([-1, 4])[:, :3]
        self.verts = v + self.trans.reshape([1, 3])
        self.G = G

    def rodrigues(self, r):
        """
        Rodrigues' rotation formula that turns axis-angle vector into rotation
        matrix in a batch-ed manner.
    
        Parameter:
        ----------
        r: Axis-angle rotation vector of shape [batch_size, 1, 3].
    
        Return:
        -------
        Rotation matrix of shape [batch_size, 3, 3].
    
        """
        theta = np.linalg.norm(r, axis=(1, 2), keepdims=True)
        # avoid zero divide
        theta = np.maximum(theta, np.finfo(np.float64).tiny)
        r_hat = r / theta
        cos = np.cos(theta)
        z_stick = np.zeros(theta.shape[0])
        m = np.dstack([
            z_stick, -r_hat[:, 0, 2], r_hat[:, 0, 1],
            r_hat[:, 0, 2], z_stick, -r_hat[:, 0, 0],
            -r_hat[:, 0, 1], r_hat[:, 0, 0], z_stick]
        ).reshape([-1, 3, 3])
        i_cube = np.broadcast_to(
            np.expand_dims(np.eye(3), axis=0),
            [theta.shape[0], 3, 3]
        )
        A = np.transpose(r_hat, axes=[0, 2, 1])
        B = r_hat
        dot = np.matmul(A, B)
        R = cos * i_cube + (1 - cos) * dot + np.sin(theta) * m
        return R

    def with_zeros(self, x):
        """
        Append a [0, 0, 0, 1] vector to a [3, 4] matrix.
    
        Parameter:
        ---------
        x: Matrix to be appended.
    
        Return:
        ------
        Matrix after appending of shape [4,4]
    
        """
        return np.vstack((x, np.array([[0.0, 0.0, 0.0, 1.0]])))

    def pack(self, x):
        """
        Append zero matrices of shape [4, 3] to vectors of [4, 1] shape in a batched
        manner.
    
        Parameter:
        ----------
        x: Matrices to be appended of shape [batch_size, 4, 1]
    
        Return:
        ------
        Matrix of shape [batch_size, 4, 4] after appending.
    
        """
        return np.dstack((np.zeros((x.shape[0], 4, 3)), x))

    def save_to_obj(self, path):
        """
        Save the SMPL model into .obj file.
    
        Parameter:
        ---------
        path: Path to save.
    
        """
        with open(path, 'w') as fp:
            for v in self.verts:
                fp.write('v %f %f %f\n' % (v[0], v[1], v[2]))
            for f in self.faces + 1:
                fp.write('f %d %d %d\n' % (f[0], f[1], f[2]))

In [12]:

# load SMPL model and set to standard pose/shape
smpl = SMPLModel('/content/drive/MyDrive/TFM_ROMP/ROMP/models/smpl/SMPL_NEUTRAL.pkl')
trans = np.zeros(smpl.trans_shape)
beta = np.zeros(smpl.beta_shape)
pose = np.zeros(smpl.pose_shape)
vert = smpl.set_params(beta=beta, pose=pose, trans=trans)


In [13]:
!mkdir /content/drive/MyDrive/TFM_ROMP/ROMP_labeling

In [14]:

# calculate face areas
smpl_face_area = []
for f in smpl.faces:
    v0, v1, v2 = vert[f[0]], vert[f[1]], vert[f[2]]
    e0 = np.linalg.norm(v0 - v1)
    e1 = np.linalg.norm(v1 - v2)
    e2 = np.linalg.norm(v2 - v0)
    s = np.sqrt((e0+e1+e2) * (e0+e1-e2) * (e1+e2-e0) * (e2+e0-e1)) * 0.25
    smpl_face_area.append(s)
np.save('/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/smpl_face_area.npy', smpl_face_area)


In [15]:

with open('/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/whole_body.obj', 'w') as fp:
    for v in smpl.verts:
        fp.write('v %f %f %f\n' % (v[0], v[1], v[2]))

# label some specific body parts according to skinning weights
def get_vertices_bound_to_jnts(skinning_weights, jnts):
    weights_of_interest = np.sum(skinning_weights[:, jnts], axis=1)
    return np.where(weights_of_interest > 0.5)

def save_part_of_smpl(smpl, vert_inds, filename):
    with open(filename, 'w') as fp:
        for vi in vert_inds:
            v = smpl.verts[vi]
            fp.write('v %f %f %f\n' % (v[0], v[1], v[2]))

In [16]:
left_hand_indices = get_vertices_bound_to_jnts(smpl.weights, [20, 22])[0]
save_part_of_smpl(smpl, left_hand_indices, '/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/left_hand.obj')
np.save('/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/left_hand_indices.npy', left_hand_indices)
right_hand_indices = get_vertices_bound_to_jnts(smpl.weights, [21, 23])[0]
save_part_of_smpl(smpl, right_hand_indices, '/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/right_hand.obj')
np.save('/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/right_hand_indices.npy', right_hand_indices)



In [17]:
left_arm_indices = get_vertices_bound_to_jnts(smpl.weights, [16, 18])[0]
save_part_of_smpl(smpl, left_arm_indices, '/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/left_arm.obj')
np.save('/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/left_arm_indices.npy', left_arm_indices)
right_arm_indices = get_vertices_bound_to_jnts(smpl.weights, [17, 19])[0]
save_part_of_smpl(smpl, right_arm_indices, '/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/right_arm.obj')
np.save('/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/right_arm_indices.npy', right_arm_indices)



In [18]:
head_indices = get_vertices_bound_to_jnts(smpl.weights, [12, 15])[0]
save_part_of_smpl(smpl, head_indices, '/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/head.obj')
np.save('/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/head_indices.npy', head_indices)



In [19]:
left_leg_indices = get_vertices_bound_to_jnts(smpl.weights, [1, 4])[0]
save_part_of_smpl(smpl, left_leg_indices, '/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/left_leg.obj')
np.save('/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/left_leg_indices.npy', left_leg_indices)
right_leg_indices = get_vertices_bound_to_jnts(smpl.weights, [2, 5])[0]
save_part_of_smpl(smpl, right_leg_indices, '/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/right_leg.obj')
np.save('/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/right_leg_indices.npy', right_leg_indices)



In [20]:
left_foot_indices = get_vertices_bound_to_jnts(smpl.weights, [7, 10])[0]
save_part_of_smpl(smpl, left_foot_indices, '/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/left_foot.obj')
np.save('/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/left_foot_indices.npy', left_foot_indices)
right_foot_indices = get_vertices_bound_to_jnts(smpl.weights, [8, 11])[0]
save_part_of_smpl(smpl, right_foot_indices, '/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/right_foot.obj')
np.save('/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/right_foot_indices.npy', right_foot_indices)



In [21]:
torso_indices = get_vertices_bound_to_jnts(smpl.weights, [0, 3, 6, 9, 13, 14])[0]
save_part_of_smpl(smpl, torso_indices, '/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/torso.obj')
np.save('/content/drive/MyDrive/TFM_ROMP/ROMP_labeling/torso_indices.npy', torso_indices)

